# Power-system State-Estimation using Gurobi 

This notebook will take you through the steps required to perform custom Weighted Least Squares State-Estimation for a few pandapower networks. 

Methodology: 



In [1]:
import gurobipy as gp 
import logging 
import os 
import pandapower as pp 
import numpy as np 
import pandas as pd 
from utils import get_branch_parameters, get_edge_index_from_ppnet, get_neighbor_dict

# configure logging
log_dir = "logs"
log_file = "script_log.txt"

# ensure log directory exists
os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    filename=os.path.join(log_dir, log_file),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

env = gp.Env(params={"LogToConsole": 0})

## Consider a 4-Bus Network: 

<img src="4_bus_network/4_bus_network.001.jpeg" width="500">

In [2]:
net = pp.create_empty_network()

bus1 = pp.create_bus(net, vn_kv=20.0, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=20.0, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")  # connected via transformer
bus4 = pp.create_bus(net, vn_kv=20.0, name="Bus 4")

# standard 20 kV line type 
std_line_type = "NAYY 4x50 SE"

# Create standard lines
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=1.0,
               std_type=std_line_type, name="Line 1-2")

pp.create_line(net, from_bus=bus1, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 1-4")

pp.create_line(net, from_bus=bus2, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 2-4")

pp.create_transformer(net, bus2, bus3, std_type="0.25 MVA 20/0.4 kV", name="Trafo 2-3")
net.trafo.shift_degree = 0.0 # bug in pandapwer 

pp.create_ext_grid(net, bus=bus2, vm_pu=1.02, name="Grid Connection")

pp.create_load(net, bus3, p_mw=0.2, q_mvar=0.05, name="Load Bus 3")
pp.create_load(net, bus4, p_mw=0.3, q_mvar=0.1, name="Load Bus 4")

pp.runpp(net)

edge_index = get_edge_index_from_ppnet(net=net)

### Initialize the variables

In [3]:
# how many buses are there? 
num_buses = len(net.bus.index)

# how many state-variables 
n = 2*num_buses - 1 # \theta_1 = 0 for reference 

# consider fully-observable network, so number of measurements >= number of states 
# using two measurements of active power and voltage at each bus 
m = 2*num_buses

# initialize model 
gp_model = gp.Model("WLS_State_Estimation")

# measurement vector as 5% tolerance to power flow results 
V_meas = np.random.normal(net.res_bus.vm_pu.values, 0.05/3)
P_meas = np.random.normal(net.res_bus.p_mw.values, 0.05/3)

# Gurobi variables for estimated voltages and angles
# subject to voltage violations and angle violations 
X_vm_pu = {i: gp_model.addVar(name=f"|V|_bus{i}") for i in range(n)}
X_va_deg = {i: gp_model.addVar(name=f"A_bus{i}") for i in range(n)}

# slack bus angle 
X_va_deg[0].LB, X_va_deg[0].UB = 0.0, 0.0 
gp_model.update()


Set parameter Username
Set parameter LicenseID to value 2654438
Academic license - for non-commercial use only - expires 2026-04-19


In [4]:
# get line and transformer parameters
# param_dict = {(i, j): [G_ij, B_ij, g_s_ij, b_s_ij, g_sh_ij, b_sh_ij]}
# TODO: add tap-changing transformers and mutual coupling between lines
param_dict = get_branch_parameters(net=net) 


In [7]:
# get neighbor_dictionary 
# neighbor_dict = {src: [neighbors]}
neighbor_dict = get_neighbor_dict(edge_index)
neighbor_dict

{0: [1, 3], 1: [0, 2, 3], 3: [0, 1], 2: [1]}

In [6]:
obj = ... 
gp_model.setObjective, setMobjective, setObjectiveN 



NameError: name 'setMobjective' is not defined

In [ ]:
import pandapower as pp
import pandapower.networks as pn
from gurobipy import Model, GRB, QuadExpr
import numpy as np

# Set up the network
net = pp.create_empty_network()
bus1 = pp.create_bus(net, vn_kv=20.0, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=20.0, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")
bus4 = pp.create_bus(net, vn_kv=20.0, name="Bus 4")

std_line_type = "NAYY 4x50 SE"
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=1.0, std_type=std_line_type)
pp.create_line(net, from_bus=bus1, to_bus=bus4, length_km=1.0, std_type=std_line_type)
pp.create_line(net, from_bus=bus2, to_bus=bus4, length_km=1.0, std_type=std_line_type)
pp.create_transformer(net, bus2, bus3, std_type="0.4 MVA 20/0.4 kV")
net.trafo.shift_degree = 0.0

pp.create_ext_grid(net, bus=bus1, vm_pu=1.02)
pp.create_load(net, bus3, p_mw=0.2, q_mvar=0.05)
pp.create_load(net, bus4, p_mw=0.3, q_mvar=0.1)

# Run power flow to get "measurements"
pp.runpp(net)

# Extract voltage magnitudes and active powers
V_meas = net.res_bus.vm_pu.values 
P_meas = net.res_bus.p_mw.values

# Define weights (you can customize these)
w_V = np.ones_like(V_meas) * 1.0  # weight for voltage error
w_P = np.ones_like(P_meas) * 1.0   # weight for power error

# Create Gurobi model
m = Model("WLS_StateEstimation")

# Gurobi variables for estimated voltages and active powers
V_est = {i: m.addVar(lb=0.9, ub=1.1, name=f"V_bus{i}") for i in range(len(V_meas))}
P_est = {i: m.addVar(lb=-1.0, ub=1.0, name=f"P_bus{i}") for i in range(len(P_meas))}

m.update()

# Build WLS objective function
obj = QuadExpr()
for i in range(len(V_meas)):
    obj += w_V[i] * (V_est[i] - V_meas[i]) * (V_est[i] - V_meas[i])
    obj += w_P[i] * (P_est[i] - P_meas[i]) * (P_est[i] - P_meas[i])

m.setObjective(obj, GRB.MINIMIZE)

# (Optional) Add physics-based constraints later

m.optimize()

# Output estimated states
for i in range(len(V_meas)):
    print(f"Bus {i+1}: V_est = {V_est[i].X:.4f}, P_est = {P_est[i].X:.4f}")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.4.0 24E263)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 0 rows, 8 columns and 0 nonzeros
Model fingerprint: 0xce0680e8
Model has 8 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [4e-01, 2e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [9e-01, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.11022302e-16
Bus 1: V_est = 1.0200, P_est = -0.5031
Bus 2: V_est = 1.0196, P_est = -0.0000
Bus 3: V_est = 1.0049, P_est = 0.2000
Bus 4: V_est = 1.0196, P_est = 0.3000


In [ ]:
V_est[0].x

1.02

In [ ]:
net.trafo.index.values

array([0])